In [ ]:
!nvidia-smi

In [ ]:
# install Image Data Augmentation
!pip install -q git+https://github.com/mjkvaak/ImageDataAugmentor
!pip install -q efficientnet

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import zipfile
import csv
import sys
import os


import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *

#models
from tensorflow.keras.applications.xception import Xception
import efficientnet.keras as efn 

# ImageDataAugmentor
from ImageDataAugmentor.image_data_augmentor import *
import albumentations as A

from sklearn.model_selection import train_test_split, StratifiedKFold

import PIL
from PIL import ImageOps, ImageFilter
#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline

print(os.listdir("../input"))
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)
print('Keras        :', tf.keras.__version__)

In [ ]:
!pip freeze > requirements.txt

# Setup

In [ ]:
# Settings

EPOCHS               = 5  # эпох на обучение
BATCH_SIZE           = 64 # уменьшаем batch если сеть большая, иначе не поместится в память на GPU
LR                   = 1e-3
VAL_SPLIT            = 0.15 # сколько данных выделяем на тест = 15%

CLASS_NUM            = 10  # количество классов в нашей задаче
IMG_SIZE             = 224 # какого размера подаем изображения в сеть
IMG_CHANNELS         = 3   # у RGB 3 канала
input_shape          = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)

DATA_PATH = '../input/'
PATH = "../working/car/" # рабочая директория


In [ ]:
# Setting seed for reproducibility
os.makedirs(PATH,exist_ok=False)

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)  
PYTHONHASHSEED = 0

# EDA / Анализ данных

In [ ]:
train_df = pd.read_csv(DATA_PATH+"train.csv")
sample_submission = pd.read_csv(DATA_PATH+"sample-submission.csv")
train_df.head()

In [ ]:
train_df.info()

In [ ]:
train_df.Category.value_counts()

In [ ]:
print('Extract images')
# Will unzip the files so that you can see them..
for data_zip in ['train.zip', 'test.zip']:
    with zipfile.ZipFile("../input/"+data_zip,"r") as z:
        z.extractall(PATH)
        
print(os.listdir(PATH))

In [ ]:
print('random sample')
plt.figure(figsize=(12,8))

random_image = train_df.sample(n=9)
random_image_paths = random_image['Id'].values
random_image_cat = random_image['Category'].values

for index, path in enumerate(random_image_paths):
    im = PIL.Image.open(PATH+f'train/{random_image_cat[index]}/{path}')
    plt.subplot(3,3, index+1)
    plt.imshow(im)
    plt.title('Class: '+str(random_image_cat[index]))
    plt.axis('off')
plt.show()

In [ ]:
image = PIL.Image.open(PATH+'/train/0/100380.jpg')
imgplot = plt.imshow(image)
plt.show()
image.size

# Data

### Data augmentation

In [ ]:
AUGMENTATIONS = A.Compose([
    A.Transpose(p=0.5),
    A.Flip(p=0.5),
    A.RandomBrightness(limit=0.2, always_apply=False, p=0.5),
    A.OneOf([
        A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3),
        A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1)
    ],p=1),
    A.GaussianBlur(p=0.05),
    A.HueSaturationValue(p=0.5),
    A.RGBShift(p=0.5),
    A.HorizontalFlip(p=0.5), # horizontally flip 50% of all images
    A.VerticalFlip(p=0.2), # vertically flip 20% of all images
    A.ShiftScaleRotate(shift_limit=0.0625, 
                       scale_limit=0.1, 
                       rotate_limit=45, 
                       interpolation=1, 
                       border_mode=4, 
                       p=0.5),
    A.ElasticTransform(),
    A.OneOf([
        A.CenterCrop(height=224, width=200),
        A.CenterCrop(height=200, width=224)],
        p=0.5),
    A.FancyPCA(alpha=0.1,always_apply=False, p=0.5),
    A.Resize(IMG_SIZE, IMG_SIZE)
])

### datagen

In [ ]:
train_datagen = ImageDataAugmentor(rescale=1./255,
                augment=AUGMENTATIONS,                
                validation_split=VAL_SPLIT,
                seed=RANDOM_SEED
                )


# wrop data in generator
train_generator = train_datagen.flow_from_directory(
    PATH+'train/',      # директория где расположены папки с картинками 
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    subset='training') # set as training data

test_generator = train_datagen.flow_from_directory(
    PATH+'train/',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    subset='validation') # set as validation data

In [ ]:
train_generator.show_data(rows=1, cols=3)

# Model

### 0. Preprocessing

In [ ]:
def freeze_layers(model, fine_tune_at):
    """ Freeze all the layers before the `fine_tune_at` layer """
    for layer in model.layers[:fine_tune_at]:
        layer.trainable =  False

def check_layers(m):
    """ Check the trainable status of the individual layers """
    for layer in model.layers:
        print(layer, layer.trainable)        

def m_compile(model):
    model.compile(loss="categorical_crossentropy", 
                optimizer=optimizers.Adam(lr=LR), 
                metrics=["accuracy"])

def m_fit(model):
    model.fit_generator(train_generator,
        steps_per_epoch = train_generator.samples//train_generator.batch_size,
        validation_data = test_generator, 
        validation_steps = test_generator.samples//test_generator.batch_size,
        epochs = EPOCHS,
        callbacks = callbacks_list)
    
        
def m_scores(model):
    scores = model.evaluate_generator(test_generator, verbose=1)
    print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
# Add checkpoint and manage Learning Rate

# add checkpoint to save model progress
checkpoint = ModelCheckpoint('best_model.hdf5', 
                             monitor = ['val_accuracy'], 
                             verbose = 1 , 
                             mode = 'max')

# https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping
earlystop = EarlyStopping(monitor = 'val_accuracy',
                          patience = 4,                          
                          restore_best_weights = True)

# https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.25,
                              patience=2,
                              min_lr=0.0000001,
                              verbose=1,
                              mode='auto')


callbacks_list = [checkpoint, earlystop, reduce_lr]

In [ ]:
# Download pre-trained model

# https://pypi.org/project/efficientnet/
efn_model = efn.EfficientNetB7(weights='imagenet', 
                                include_top=False, 
                                input_shape = input_shape)

In [ ]:
#efn_model.summary()

In [ ]:
# Create the model
model=Sequential()
model.add(efn_model) # Add the efn convolutional base model

# Add new layers
model.add(GlobalAveragePooling2D())

"""
model.add(Conv2D(16, kernel_size=3, strides=1, padding="same", input_shape=(32, 32, 3) ))
model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size=3, strides=1, padding="same"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))    
model.add(Dropout(0.25))

model.add(Conv2D(32, kernel_size=3, strides=1, padding="same"))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=3, strides=1, padding="same"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
"""

model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(.25))
model.add(Dense(CLASS_NUM, activation='softmax'))

### 1. Freeze the pre-trained model

In [ ]:
# Freeze all the layers
for layer in efn_model.layers[:]:
    layer.trainable = False
#check_layers(efn_model)

In [ ]:
m_compile(model)
m_fit(model)
model.load_weights('best_model.hdf5')
m_scores(model)

### 2. defrost 50% of the pre-trained model weights

In [ ]:
# https://keras.io/guides/transfer_learning/
efn_model.trainable = True
fine_tune_at = len(efn_model.layers)//2 # Fine-tune from this layer onwards

EPOCHS = 10
BATCH_SIZE = 32
LR  = 1e-4

freeze_layers(efn_model, fine_tune_at) # freeze 50%

In [ ]:
m_compile(model)
m_fit(model)
model.load_weights('best_model.hdf5')
m_scores(model)

### 3. complete weights defrosting

In [ ]:
efn_model.trainable = True

EPOCHS = 10
BATCH_SIZE = 16
LR = 1e-5

In [ ]:
m_compile(model)
m_fit(model)
model.load_weights('best_model.hdf5')
m_scores(model)

### 4. change image size

In [ ]:
efn_model.trainable = True
EPOCHS = 10
BATCH_SIZE = 4
LR = 1e-5
IMG_SIZE = 512

In [ ]:
m_compile(model)
m_fit(model)
model.load_weights('best_model.hdf5')
m_scores(model)

# Submission

In [ ]:
test_datagen = ImageDataAugmentor(rescale=1./255)

test_sub_generator = test_datagen.flow_from_dataframe( 
    dataframe=sample_submission,
    directory=PATH+'test_upload/',
    x_col="Id",
    y_col=None,
    shuffle=False,
    class_mode=None,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,)

In [ ]:
test_sub_generator.reset()
predictions = model.predict_generator(test_sub_generator, steps=len(test_sub_generator), verbose=1) 
predictions = np.argmax(predictions, axis=-1) #multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v
predictions = [label_map[k] for k in predictions]

In [ ]:
filenames_with_dir=test_sub_generator.filenames
submission = pd.DataFrame({'Id':filenames_with_dir, 'Category':predictions}, columns=['Id', 'Category'])
submission['Id'] = submission['Id'].replace('test_upload/','')
submission.to_csv('submission.csv', index=False)
print('Save submit')

In [ ]:
submission.head()

In [ ]:
# Clean PATH
import shutil
shutil.rmtree(PATH)

# Что можно сделать, чтоб улучшить результат:
* Подобрать LR, optimizer, loss
* Добавить аугментацию
* Поиграться с архитектурой
* Подобрать другие переменные (размер картинки, батч и тп)
* Добавить политику обучения
* Добавить TTA
* Найти и обучиться на других внешних данных
* Построить ансамбль из разных архитектур

### Удачи в соревновании!